In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import pickle

In [10]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [11]:
# Preprocess data
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [12]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [13]:
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=[onehot_encoder_geo.get_feature_names_out(['Geography'])])
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [14]:
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,"(Geography_France,)","(Geography_Germany,)","(Geography_Spain,)"
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [15]:
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [16]:
X_train.columns = ['_'.join(map(str, col)) if isinstance(col, tuple) else str(col) for col in X_train.columns]
X_test.columns = ['_'.join(map(str, col)) if isinstance(col, tuple) else str(col) for col in X_test.columns]


In [17]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [19]:
model = Sequential([    
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1) # by default linear activation will be applied since its used for regression
    ])

model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

model.summary()

d:\VS Code\Python\Gen AI Playlist\Venv Folder\ann churn project\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# Setup tensorBoard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
log_dir = 'regressionlogs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [21]:
# Setup Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [22]:
# Train the model
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), 
    epochs=50, 
    callbacks=[early_stopping_callback, tensorflow_callback]
)

Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 100372.0547 - mae: 100372.0547 - val_loss: 98488.1250 - val_mae: 98488.1250
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 99493.5312 - mae: 99493.5312 - val_loss: 96700.5391 - val_mae: 96700.5391
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 96388.6875 - mae: 96388.6875 - val_loss: 92160.6094 - val_mae: 92160.6094
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 90338.9922 - mae: 90338.9922 - val_loss: 84747.7188 - val_mae: 84747.7188
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 81721.0156 - mae: 81721.0156 - val_loss: 75518.0312 - val_mae: 75518.0312
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 72106.5703 - mae: 72106.5703 - val_loss: 66424.9453 - val_mae: 66424.9453
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 63347.8945 - mae: 63347.8945 - val_loss: 59015.1758 - val_mae: 59015.1758
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 

In [23]:
test_loss, test_mae = model.evaluate(X_test,y_test)
print(f'Test MAE: {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 50377.4062 - mae: 50377.4062
Test MAE: 50377.40625


In [24]:
model.save('regression_model.h5')

## Hyperparameter Tuning

### 1. Start simple
Use 1-2 hidden layers initially.
For many problems, a single hidden layer with enough neurons can approximate any function (Universal Approximation Theorem).
Deep networks (more layers) are better for complex features like images, speech, or text.

### 2. Number of neurons per layer
Typically, start with a number of neurons between input layer size and output layer size.
Common heuristic: number of neurons in a layer between (input features + output classes) / 2 and 2 × input features.
Or just start with 32, 64, 128 neurons and tune from there.

### 3. Avoid too few neurons
Too few neurons → underfitting (model too simple to learn underlying patterns).

### 4. Avoid too many neurons
Too many neurons → overfitting (model memorizes training data, poor generalization).
Also increases training time and computational cost.

### 5. Use cross-validation and hyperparameter tuning
Try different architectures and measure validation performance.
Grid search, random search, or Bayesian optimization can help tune number of layers and neurons.

### 6. Consider problem complexity
Simple tabular data/classification → shallow networks (1-2 layers).
Complex data (images, audio) → deeper networks (4+ layers or specialized architectures like CNNs).

### 7. Regularization matters
Use dropout, L2 regularization, early stopping to prevent overfitting with bigger networks.

### 8. Layer size typically decreases
Often, the first hidden layer is the largest.
Subsequent hidden layers may have fewer neurons (funnel shape).
E.g., Input (100) → Hidden1 (64) → Hidden2 (32) → Output (10).


In [25]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import pandas as pd

In [26]:
data = pd.read_csv('Churn_Modelling.csv')

## Preprocess data
### Drop irrelevant columns
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
# Encode categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

# Inorder to encoder columns with multiple categories we will use OneHotEncoding insted of LabelEncoder
# Geography column
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])

onehot_encoder_geo.get_feature_names_out(['Geography'])
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

# If you wish to directly do OneHotEncoding
# data = pd.get_dummies(data, columns=['Geography'])
# data.head()
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()
# Save encoder and scaler
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)
with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)
data.head()
# Split data into X, y
X = data.drop('Exited', axis=1)
y = data['Exited']

# Split the data into training and test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) # fit_trainsform the fit means we calculate parameters from X_trian only and transform uses those parameters to scale features
X_test = scaler.transform(X_test) # transform transforms the previosuly learned features in fit_transform

In [33]:
# Define a function to create the model and try different parameters
def create_model(neurons=32, layers=1):
    model=Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [34]:
# Create keras classifier
model = KerasClassifier(neurons=32,layers=1,build_fn=create_model, epochs=50, batch_size=10, verbose=0)

In [35]:
# Define gridsearch params
param_grid = {
    'neurons': [16,32,64,128],
    'layers': [1,2,3],
    'epochs': [50,100]
}

In [36]:
# Perform gird search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# Print the best params
print('Best: %f using %s'%(grid_result.best_score_, grid_result.best_params_))

KeyboardInterrupt: 